In [33]:
from pyspark.sql import SparkSession
import os
import json
from datetime import timedelta,datetime
from pyspark.sql.functions import *
import time

In [34]:
spark = SparkSession \
    .builder \
    .getOrCreate()

In [35]:
base_path = '/home/leavmir/data/json'
override_config_key = 'spark.override.date'
parquet_write_path = '/home/leavmir/data/parquet/'
current_date = datetime.now().strftime("%Y-%m-%d")

In [36]:
try: 
    conf_value = spark.conf.get(override_config_key) 
    paths_to_read = '{}/date={}'.format(base_path,conf_value)
except:
    print("Parameter is not set")
    paths_to_read = '{}/date={}'.format(base_path,current_date)


Parameter is not set


In [37]:
df = spark.read.option("basePath","/home/leavmir/data/json") \
    .option("multiLine", "True") \
        .json(paths_to_read)

In [38]:
json_column = df.select("publishedAt","source")
#json_column.printSchema()
exploded_json_column = json_column.select("source.*","*").drop("source")

In [39]:
final_dataframe = df.join(exploded_json_column, "publishedAt") \
    .withColumn("timestamp",to_timestamp("publishedAt")) \
    .drop("id","title","urlToImage","source","publishedAt")

In [40]:
# parquet_path = '/home/leavmir/data/parquet/{}' \
#     .format(time.strftime("date=%Y-%m-%d"))
# if not os.path.isdir(parquet_path):
#     os.mkdir(parquet_path)

In [41]:
final_dataframe = df.join(exploded_json_column, "publishedAt") \
    .withColumn("timestamp",to_timestamp("publishedAt")) \
    .drop("id","title","urlToImage","source","publishedAt")

In [42]:
final_dataframe.write.mode("overwrite") \
    .partitionBy("date") \
    .option("partitionOverwriteMode","dynamic") \
    .parquet(parquet_write_path)